Определите количество мужчин и женщин среди испытуемых. Обратите внимание, что способ кодирования переменной gender мы не знаем. Воспользуемся медицинским фактом, а именно: мужчины в среднем выше женщин.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();
df = pd.read_csv('../input/cardiovascular-disease-dataset/cardio_train.csv', sep=';')
df.head()
new_df = pd.DataFrame()
new_df['mean_height'] = df.groupby('gender')['height'].mean()
new_df['count'] = df['gender'].value_counts()
new_df

Верно ли, что мужчины более склонны к употреблению алкоголя, чем женщины?

In [ ]:
pd.crosstab(df['gender'], df['alco'])

Каково различие между процентами курящих мужчин и женщин?

In [ ]:
pd.crosstab(df['gender'], df['smoke'], normalize=True)
df[df['gender'] == 1]['smoke'].sum() / df[df['gender'] == 1].shape[0]
df[df['gender'] == 2]['smoke'].sum() / df[df['gender'] == 1].shape[0]

Какова разница между средними значениями возраста для курящих и некурящих?

In [ ]:
df['age'] = (df['age'] / 365).round()
df.groupby('smoke')['age'].mean()

Создайте новый признак --- BMI (body mass index, индекс массы тела). Для этого разделите вес в килограммах на квадрат роста в метрах. Считается, что нормальные значения ИМТ составляют от 18.5 до 25. Выберите верные утверждения:

(a) Средний ИМТ находится в диапазоне нормальных значений ИМТ.

(b) ИМТ для женщин в среднем выше, чем для мужчин.

(c) У здоровых людей в среднем более высокий ИМТ, чем у людей с ССЗ.

(d) Для здоровых непьющих мужчин ИМТ ближе к норме, чем для здоровых непьющих женщин

In [ ]:
df['BMI'] = df['weight'] / ((df['height']/100)**2)
df['BMI'].mean()
df.groupby('gender')['BMI'].mean()
df.groupby('cardio')['BMI'].mean()
df.pivot_table(values=['BMI'], index=['cardio', 'alco', 'gender'], aggfunc='mean')

Удалите пациентов, у которых диастолическое давление выше систолического. Какой процент от общего количества пациентов они составляли?

In [ ]:
df[df['ap_hi'] < df['ap_lo']].shape[0] / df.shape[0]
df.drop(df[df['ap_hi'] < df['ap_lo']].index)

На сайте Европейского общества кардиологов представлена шкала SCORE. Она используется для расчёта риска смерти от сердечно-сосудистых заболеваний в ближайшие 10 лет.

Рассмотрим верхний правый прямоугольник, который показывает подмножество курящих мужчин в возрасте от 60 до 65 лет (значения по вертикальной оси на рисунке представляют верхнюю границу).

Мы видим значение 9 в левом нижнем углу прямоугольника и 47 в правом верхнем углу. Это означает, что для людей этой возрастной группы с систолическим давлением менее 120 и низким уровнем холестерина риск сердечно-сосудистых заболеваний оценивается примерно в 5 раз ниже, чем для людей с давлением в интервале [160, 180] и высоким уровнем холестерина.

Вычислите аналогичное соотношение для наших данных.

In [ ]:
platform_genre_sales = df[(df['ap_hi'] > 120) & (df['ap_hi'] < 180) & (df['age'] > 60) & (df['age'] < 65) & (df['gender'] == 2)].pivot_table(
                        index='ap_hi', 
                        columns='cholesterol', 
                        values='cardio', 
                        aggfunc=sum).fillna(0).applymap(float)
f, ax = plt.subplots(figsize=(10, 17))
sns.heatmap(platform_genre_sales, annot=True, linewidths=.5, fmt=".1f", ax=ax, yticklabels=5)

Визуализируйте распределение уровня холестерина для различных возрастных категорий.

In [ ]:
plt.figure(figsize=(25, 15))
sns.countplot(y='age', hue='cholesterol', data=df);

Как распределена переменная BMI? Есть ли выбросы

In [ ]:
sns.boxplot(df['BMI']);

Как соотносятся ИМТ и наличие ССЗ? Придумайте подходящую визуализацию.

In [ ]:
df.groupby(['cardio', 'gender'])['BMI'].mean().plot(kind='bar') 
plt.ylabel('BMI') 
plt.show();